In [3]:
from concurrent.futures import ProcessPoolExecutor
import concurrent.futures
import itertools
import multiprocessing

In [26]:
class TestClass:

    def __init__(self, thing1, thing2, nproc=1):
        self.thing1 = thing1
        self.thing2 = thing2
        self.nproc = nproc

    def combine(self):
        self.combined = self.thing1 + self.thing2
        if self.nproc > 1:
            return self

In [27]:
things = 'a,b,c,d,e,f,g'.split(',')
combos = [x for x  in itertools.combinations(things,2)]

In [28]:
class_lst = []
for th in combos:
    tc = TestClass(thing1=th[0],thing2=th[1])
    class_lst.append(tc)

In [29]:
class_lst[0].__dict__

{'thing1': 'a', 'thing2': 'b', 'nproc': 1}

In [30]:
for tc in class_lst:
    tc.combine()

In [31]:
class_lst[0].__dict__

{'thing1': 'a', 'thing2': 'b', 'nproc': 1, 'combined': 'ab'}

In [32]:
class_lst = []
for th in combos:
    tc = TestClass(thing1=th[0],thing2=th[1],nproc=12)
    class_lst.append(tc)

In [33]:
with ProcessPoolExecutor(max_workers=12,
                         mp_context=multiprocessing.get_context('fork')) as exe:
    out = []
    for i,tc in enumerate(class_lst):
        fs = exe.submit(tc.combine)
        fs.ind = i
        out.append(fs)
    outs = [None] * len(out)
    for f in concurrent.futures.as_completed(out):
        outs[f.ind] = f.result()

In [35]:
outs[0].__dict__

{'thing1': 'a', 'thing2': 'b', 'nproc': 12, 'combined': 'ab'}

In [36]:
class_lst[0].__dict__

{'thing1': 'a', 'thing2': 'b', 'nproc': 12}

In [4]:
from architector import convert_io_molecule
from architector.io_calc import CalcExecutor

In [5]:
mols = ['C','N','O','CN','COP']

In [18]:
def pprocess(inmol):
    mol = convert_io_molecule(inmol)
    thing = CalcExecutor(mol, relax=True)
    thing.mol.ase_atoms.calc = None
    return thing

In [14]:
thing = pprocess('CC')

In [19]:
with ProcessPoolExecutor(max_workers=12,
                         mp_context=multiprocessing.get_context('fork')) as exe:
    out = []
    for i,mol in enumerate(mols):
        fs = exe.submit(pprocess,mol)
        fs.ind = i
        out.append(fs)
    outs = [None] * len(out)
    for f in concurrent.futures.as_completed(out):
        outs[f.ind] = f.result()